In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import log_loss

import gc
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers

from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import callbacks
import tensorflow as tf

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv("../input/tabular-playground-series-jun-2021/test.csv")
submission = pd.read_csv("../input/tabular-playground-series-jun-2021/sample_submission.csv")
submission = submission.set_index('id')

In [ ]:
x_cols = train.columns[0:-1].tolist()
y_col = train.columns[-1]

In [ ]:
x_cols

In [ ]:
y_col

In [ ]:
x_cols

In [ ]:
x_cols=x_cols[1:]
x_cols

In [ ]:
X=train[x_cols]
y=train[y_col]

In [ ]:
class_map = {'Class_1': 0,
            'Class_2': 1,
            'Class_3': 2,
            'Class_4': 3,
            'Class_5': 4,
            'Class_6': 5,
            'Class_7': 6,
            'Class_8': 7,
            'Class_9': 8}
y = y.map(class_map).astype('int')
y

In [ ]:
#Converting target series to matrix for multiclass classification on Keras

targets = to_categorical(y)
targets

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#Splitting of dataset into Training set and testing set (80% and 20% respectively)
X_train, X_test, Y_train, Y_test = train_test_split(X, targets, test_size=0.2, random_state=42)

In [ ]:
conv_inputs = layers.Input(shape = (75))
#----------- Embedding layers ----------------------
embed = layers.Embedding (input_dim = 1024, 
                          output_dim = 7,
                          embeddings_regularizer='l2')(conv_inputs)
#----------- Convolution layers ----------------------
embed = layers.Conv1D(128,1,activation = 'relu')(embed)        
embed = layers.Flatten()(embed)
hidden = layers.Dropout(0.5)(embed)

#----------- Residual blocks layers ----------------------
hidden = tfa.layers.WeightNormalization(
            layers.Dense(
            units=64,
            activation ='selu',
            kernel_initializer = "lecun_normal"))(hidden)

output = layers.Dropout(0.3)(layers.Concatenate()([embed, hidden]))
output = tfa.layers.WeightNormalization(
layers.Dense(
            units = 32,
            activation='relu',
            kernel_initializer = "lecun_normal"))(output) 
output = layers.Dropout(0.4)(layers.Concatenate()([embed, hidden, output]))
output = tfa.layers.WeightNormalization(
layers.Dense(
            units = 16, 
            activation = 'elu',
            kernel_initializer = "lecun_normal"))(output)

#----------- Final layer -----------------------
conv_outputs = layers.Dense(
            units = 9, 
            activation ='softmax',
            kernel_initializer ="lecun_normal")(output)

#----------- Model instantiation  ---------------
model = Model(conv_inputs,conv_outputs)

In [ ]:
K.clear_session()
cce = keras.losses.CategoricalCrossentropy(name = 'cat_crossentr')

def custom_metric(y_true, y_pred):
    y_pred = K.clip(y_pred, 1e-15, 1-1e-15)
    loss = cce(y_true, y_pred)
    return loss

In [ ]:
OOF_PRED = np.zeros((X_train.shape[0], 9))
TEST_PRED = np.zeros((X_test.shape[0], 9))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(learning_rate=2e-4), metrics=custom_metric)

In [ ]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True)
plateau = callbacks.ReduceLROnPlateau(monitor='val_loss',
    factor=0.5,
    patience=2,
    verbose=0,
    mode='auto',
    min_delta=0.0001,
    cooldown=0,
    min_lr=0) 

In [ ]:
from keras.callbacks import CSVLogger, ModelCheckpoint
script_name_base = "TPS_KERAS_"
model_checkpoint = ModelCheckpoint('{:}.epoch{{epoch:02d}}-train{{loss:.5f}}-val{{val_loss:.5f}}.hdf5'.format(script_name_base),
                                    save_best_only=True)

In [ ]:
import os
import glob

files = glob.glob('./*')
for f in files:
    os.remove(f)

In [ ]:
model.fit(X_train, Y_train,batch_size = 32, epochs = 500,validation_data=(X_test, Y_test),callbacks=[early_stopping,plateau,model_checkpoint],verbose = 1)

In [ ]:
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
del model

In [ ]:
conv_inputs = layers.Input(shape = (75))
#----------- Embedding layers ----------------------
embed = layers.Embedding (input_dim = 1024, 
                          output_dim = 7,
                          embeddings_regularizer='l2')(conv_inputs)
#----------- Convolution layers ----------------------
embed = layers.Conv1D(128,1,activation = 'relu')(embed)        
embed = layers.Flatten()(embed)
hidden = layers.Dropout(0.5)(embed)

#----------- Residual blocks layers ----------------------
hidden = tfa.layers.WeightNormalization(
            layers.Dense(
            units=64,
            activation ='selu',
            kernel_initializer = "lecun_normal"))(hidden)

output = layers.Dropout(0.3)(layers.Concatenate()([embed, hidden]))
output = tfa.layers.WeightNormalization(
layers.Dense(
            units = 32,
            activation='relu',
            kernel_initializer = "lecun_normal"))(output) 
output = layers.Dropout(0.4)(layers.Concatenate()([embed, hidden, output]))
output = tfa.layers.WeightNormalization(
layers.Dense(
            units = 16, 
            activation = 'elu',
            kernel_initializer = "lecun_normal"))(output)

#----------- Final layer -----------------------
conv_outputs = layers.Dense(
            units = 9, 
            activation ='softmax',
            kernel_initializer ="lecun_normal")(output)

#----------- Model instantiation  ---------------
model = Model(conv_inputs,conv_outputs)

In [ ]:
import glob
import os

list_of_files = glob.glob('./*.hdf5') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

In [ ]:
model.load_weights(latest_file)

In [ ]:
pred=model.predict(X_train)
pred1=model.predict(X_test)

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
log_loss(Y_train, pred)

In [ ]:
log_loss(Y_test, pred1)

In [ ]:
test

In [ ]:
submission

In [ ]:
test.drop('id',axis=1,inplace=True)

In [ ]:
pred_test=model.predict(test)

In [ ]:
submission.iloc[:, :] = pred_test
submission

In [ ]:
submission=submission.reset_index()

In [ ]:
submission.to_csv('Model_predict.csv', index = False)